"Деревянные модели не требуют нормировки по построению" - Радослав Нейчев

## Recap того что было

если не надо что-то на вероятность похожее, то tanh лучше предпочесть сигмоиде, а лучше просто релу юзать

бэкпроп просто идет из конца в начало

**Оптимизаторы:**

батч - случайная подвыборка датасета (как правило берут степени двойки)

Если грузить весь датасет и не делить на батчи, то у нас как бы один батч - это весь датасет и мы будем весы обновлять лишь 1 раз за эпоху, когда пройдем весь датасет, короче такое обучение будет просто нереально долгим, плюс грузить весь датасет в оперативную память почти никогда нельзя

SGD подразумевают именно по батчам - считаем loss для батча - не среднее функций потерь по всем элементам датасета, а для батча, лосс - среднее функций потерь по всем элементам батча (но получается довольно-таки шумная заивисимость лосса от итерации)

Ниже: лосс - среднее значение функции потерь на батче (или всем датасете, если на батчи не делить)

- Короче лосс - это усреднение функции потерь по выборке

- С чем я затупил: каждый объект батча - это наша строчка из датасета, то есть там есть все признаки у каждого объекта, тогда мы можем для каждого такого объекта посчитать функции потерь, потом их сложить и поделить на размер батча - получим наш лосс. Для каждого признака у нас своя производная функции потерь по весу для этого признака. Короче суть в том, что значение производной функции потерь зависит от нашего предсказания $\hat{y_i}$ для каждого из объектов батча, а так как у нас всего 4 объекта, то значение производной лосса для какого-то веса будет отличаться от того, которое мы полчили бы, если бы считали лосс как среднее функции потерь по всему датасету, от этого и появляются шумы.

В таком вычислении градиента могут попасться батчи из зашумленных элементов или чисто случайно выдернуть только изображения котов, например, а у нас там еще куча изображений другой природы

Шумовые объекты - у них неадекватная связь между значениями признаков и действительным значением целевой переменной ( y_true )

короче, видны минусы и плюсы SGD

## Методы оптимизации для нейронных сетей

### Доработка SGD

W - тензор из вообще всех параметров, какие есть у сетки

$L_i$ - это функция потерь для i объекта выборки

L - здесь сам лосс по N элементной выборке

$x_i$ - вектор признаков для i объекта на входе в сетку

$y_i$ - предсказание для i объекта на выходе из сетки ($y_i = \hat{f}(x_i, W)$)

$$
L(W) = \dfrac{1}{N}\sum_{i=1}^N L_i(x_i, y_i, W)
$$

L - это скаляр, а диффернцировать скаляр по тензору очень просто - это тензор такого же размера, а в каждой ячейке стоит частная производная по нужному весу,

$\nabla_W L(W)$ - обозначим как раз так тензор из частных производных по всем весам, ранее я осознал, что $\nabla_W L(W) = g(X, Y, W)$ - он зависит от совокупности всех признаков, всех выходов и от значений весов до обновления он тоже зависит (зависимость от значения признаков всех объектов и значений предсказаний для всех объектов как раз-таки делает SGD немного шумным), теперь вообще станет понятно, почему это так:

$$
\nabla_W L(W) = \dfrac{1}{N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)
$$

$\nabla_W L_i(x_i, y_i, W)$ - это тоже тензор, где в каждой клетке лежит значение частной производной по весу

Комментарий по поводу формулы: Тензор $\nabla_W L(W)$ из частных производных - в нем в каждой клетке лежит среднее значение частной производной функции потерь по этому весу

- Ну и понятно, что усреднение по маленьким батчам ведет к зашумленному градиенту для L

#### Идея по улучшению: SGD with momentum

- Он позволяет проскакивать локальные минимумы
- Работает супер быстро

Как это можно мыслить - считаем, что лосс обладает некоторой инерцией и он накапливает импульс, когда скатывается и моментально обнулиться или резко изменить значение он не сможет

https://youtu.be/6CvpMOO-DB4?t=877

- Мы считаем, что на наш точку в пространстве параметров модели действует сила - градиент лосса как раз (мыслим лосс как потенциал):

$x_t$ - тензор параметров модели в данный момент ( $x_0$ инициализурется банально стратовыми весами )

$v_t$ - это некоторая скорость, которую успели набрать параметры модели - они теперь инерционные ( $v_0$ теперь надо инициализировать тоже )

Создадим инерционность системы: (в начальный момент нужно просто $v_0$ инициализировать, так как теперь у нас два вектора задают траекторию)

$$
v_{t+1} = \rho v_t + \nabla f(x_t) \text{ - аналог 2 закона Ньютона}
$$

$$
x_{t+1} = x_t - \alpha \cdot v_{t+1}
$$

- По сути у нас здесь делают вклад значения предыдущих смещений, то есть наш сдвиг - векторная сумма из накопленного и текущего, но тогда если мы уже сделали
целую кучу шагов, то может так случиться, что нас очень далеко унесет мимо, есть решение:

- Nesterov momentum:
  
Сначала смещаемся по velocity, а потом cчитаем градиент уже для сдвинутого по velocity положения параметров и оттуда делаем сдвиг на градиент: https://youtu.be/6CvpMOO-DB4?t=1004

**Есть две главные проблемы, которые моментум пытается решить:**

- Попадание в глобальный минимум
- Неустойчивость градиента по батчам

### Adagrad: SGD with cache

#### Подбор learning rate:

Возникает идея сделать learning rate разным для разных параметров

Случается так, что некоторые параметры скачут туда-сюда, для них lr слишком большой, а другие наоборот почти не меняются, для них очень маленький lr.

Это можно решить, если как-то отслеживать как у нас менялась производная по этому параметру на предудущих шагах, 

$x_t$ - тензор параметров на t шаге

$$
cache_{t+1} = cache_t + (\nabla f(x_t))^2 \text{ тут поэлементный квадрат для тензора}
$$

Мы как бы для каждого параметра собираем то, какое значение принимает производная по этому параметру и кладем все это в кэш

Мы прибавляем тензор с квадратми частных производных, потому что нас волнует здесь именно абсолютное значение

Как теперь обновлять параметры:

$$
x_{t+1} = x_t - \alpha \cdot \dfrac{\nabla f(x_t)}{cache_{t+1}+\epsilon}        \text{ тут поэлементное частное для тензора}
$$

Логика: если для конкретного параметра значения из кэша маленькое, значит градиент примерно постоянный, значит он не шумит, а значит можно позволить ему двигаться быстрее. Если наоборот, то значит градиент скачет, а значит значение в кэша накопится и он будет поспокойнее двигаться

- Короче мы для каждого параметра имеем какой-то свой нормировочный коэффициент на каждом шагу

Но кэш всегда будет увеличиваться, а значит у нас в среднем будет уменьшаться размер шага вообще для всех - и так мы можем вообще не попасть в нужный нам минимум

#### RMSprop - решение этой проблемы

RMSprop: SGD with cache with exp smoothing:

$$
cache_{t+1} = \beta cache_t + (1-\beta)(\nabla f(x_t))^2 \text{ тут поэлементный квадрат для тензора}
$$

Таким образом, старые шаги постепенно начинают уменьшать свой вклад в кэш и основной вклад в кэш делают последние шаги

$$
x_{t+1} = x_t - \alpha \cdot \dfrac{\nabla f(x_t)}{cache_{t+1}+\epsilon}        \text{ тут поэлементное частное для тензора}
$$

- Но ведь идеи инерции и подбора lr друг другу не противоречат, почему бы их не объединить?

### Adam

Adam - это мы накапливаем инерцию плюс кэш, который позволяет нам нормировать вклад для каждого параметра: https://youtu.be/6CvpMOO-DB4?t=1725

$\alpha$ все равно надо руками задавать

#### Learning rate decay

Можно 3e-4 использовать для лернинг рейта, если датасет сложный +-

- learning rate все равно приходится выбирать! Никакие нормировочные коэффициенты тут не спасут

- Можно смотреть когда у нас Loss выходит на плато и тогда уменьшать learning rate - это learning rate decay - его надо использовать

Лучше всего смотреть на качество и на обучающей и на валидационной

## Нормализация в глубоком обучении

Вся аргументация для регуляризации параметров и нормализации данных берется из закона больших чисел и центральной предельной теоремы

- регуляризация нужна для того, чтобы модель не переобучалась

Для линейных моделей l1 используется в предположении распределения весов по Лапласу, l2 - по нормальному распределению. То бишь мы внедряем l2 регуляризацию в предположении, что короткие векторы весов более вероятны, чем длинные.

- Мы сделали предположение - мы его внедрили в качестве члена в функции потерь

Если качество на трейне постепенно увеличивается, а на тесте нет, то мы переобучаемся

### Нормализация данных

Для линейной модели мы предполагаем, что шум распределен тем или иным образом с центром в нуле, в зависимости от предположения для распределения шума, мы выбираем функцию потерь. То есть для максимизации правдоподобия нашего предсказания нам надо минимизировать именно квадрат отклонения, если у нас шум распределен нормально.

Нормализация нужна для того, чтобы модель была менее чувствительна к небольшим изменениям параметров и чтобы параметры было легче оптимизировать. Если у нас нулевое среднее и единичная дисперсия, то все обучение будет более устойчивым

Без нормализации, в линейных моделях регуляризация вообще не будет работать, потому что тогда наши предположения не работают

коротко:

- Линейная модель не хочет получать на вход неотнормированные данные

**Появление идеи о том, что надо нормировать перед каждым линейным слоем:**

- Проблема со скрытыми слоями:

Второй слой не имеет понятия о том, что он второй, он просто получает в себя какое-то признаковое описание

### Batch normalization

- Проблема сдвига переменных:

Когда мы обновляем весы для 2 слоя, мы опираемся на то, что в него пришло, а затем, обновив весы для этого слоя мы проходим назад и обновляем весы для предыдущего слоя, таким образом, мы меняем то, что теперь должно приходить на 2 слой, то есть мы немного ломаем нашу модель

То есть мы сначала обновили второй при условии текущего конкретного первого, но потом мы поменяли первый и таким образом, сломали второму жизнь.

Раньше с этим боролись, применяя маленький learning rate, то есть мы типо слабо обновляли предыдущий слой и как бы поэтому то, что выдает 2 слой +- правильно

- Как написано у Николаенко:
1) Forward pass: второму слою передаем некоторый вход при некотором векторе весов первого слоя, затем выдаем выход 2 слоя
2) Backward pass: используя выход 2 слоя, обновляем весы 2 слоя при условии текущего первого слоя. Затем мы обновляем весы 1 слоя и получается, что у нас изменяется выход первого слоя и все чему мы только что обучили 2 слой было просто бессмысленно.

- В качестве решения предлагается нормализация данных после каждого слоя, но нельзя просто так взять и нормализовать данные, от этого наш сдвиг никуда не уйдет. Чтобы все работало, нам надо учитывать нормализацию в backpropagation. То есть мы втыкаем новый нормализационный слой.

- Полноценно по-честному мы не можем нормализовать после каждого выхода, но мы можем делать так:

$$
h_i = \dfrac{h_i-\mu_i}{\sqrt{\sigma_i^2}}
$$

Каждый признак для нового признакового представления приводим к нулевому среднему и единичной дисперсии

Во время обучения будем запоминать среднее и дисперсию, чтобы потом можно было дать предсказание:

$$
\mu_i := \alpha \cdot mean_{batch} + (1-\alpha)\cdot \mu_i
$$

$$
\sigma^2_i := \alpha \cdot variance_{batch} + (1-\alpha)\cdot \sigma^2_i
$$

Сам алгоритм: В нем мы обучаем параметры: вектор из бета, вектор из гамма - для всех признаков

![alt text](./BN.png "Title")

Здесь показано на примере одного признака $x$. Так же делаем со всеми признаками на выходе.

Короче считаем сигма и мю по батчу для всех компонент, потом скейлим все компоненты и применяем линейное преобразование. Для каждой компоненты у нас свои бета и гамма

Че делает скейл энд шифт: может быть в среднем и дисперсии уже содержалась какая-то информация, а мы взяли и убрали их просто, гамма и бета позволяют нам не терять эту информацию. Так же мы хотим, чтобы этот слой умел делать тождественное преобразование

- Тут чинится Проблема сдвига переменных, потому что у нас на каждом слое одинаковое распределение имеют признаки

### Регуляризация в deep learning

L1, L2, elastic net:

Точно так же добавляем дополнительное слагаемое в loss. Это нужно для того, что если у нас были наименьшие из доступных параметров среди всех линейно зависимых выборок из параметров. Это нужно для того, чтобы модель была устойчивой и не было переобучения.

Делаем $+ \lambda R(W)$, где $R(W)=||W||_1$ или 2 норма или $R(W)=\beta||W||_2^2+(1-\beta)||W||_1$ 

#### Dropout

Это отдельная техника регуляризации именно для нейронок, это регуляризация уже на уровне архитектуры модели

- Мы зануляем некоторые признаки на некторых слоях и делаем предсказание таким образом

- Дропаут используется только при обучении модели, при валидации мы естественно уже используем все нейроны в полную силу

То есть мы обучаем сетку, зануляя на каждом слое случайные нейроны, таким образом мы делаем ее менее чувствительной к данным, теперь соседние нейроные как бы подхватывают на себя задачу занулившихся

As a neural network learns, neuron weights settle into their context within the network. Weights of neurons are tuned for specific features providing some specialization. Neighboring neurons become to rely on this specialization, which if taken too far can result in a fragile model too specialized to the training data. This reliant on context for a neuron during training is referred to complex co-adaptations.

You can imagine that if neurons are randomly dropped out of the network during training, that other neurons will have to step in and handle the representation required to make predictions for the missing neurons. This is believed to result in multiple independent internal representations being learned by the network.

The effect is that the network becomes less sensitive to the specific weights of neurons. This in turn results in a network that is capable of better generalization and is less likely to overfit the training data.

Dropout можно рассматривать как ансамблирование разных нейронных сетей, потому что на каждом шаге мы обучаем лишь подсеть

Мы платим за устойчивость способностью решать более сложные задачи

#### Data augmentatuin

Это еще один метод регуляризации

- У всех регуляризаций есть цена за ее применение - например, для л1 и л2 это смещение относительно решения, которые мы имщем.

- Регулризация - это внесение некоторой информации в модель, которую мы знаем до начала обучения